# Introduction

If you were to generate an image for every possible arrangement of pixels, then you would have every single image that has ever been created, _or ever will_.

Could you dig out images from the past?

_From the future?_

Could you find people who never existed?

_Could you find yourself?_

# A working model

In this model, each pixel contains a single integer value which denotes its color.

For example, an 2x2 image with 2 colors may look like this:

    00
    00

There are 16 possible permutations for this image, the other 15 are:

    10  01  11  00  10  01  11  00  10  01  11  00  10  01  11
    00  00  00  10  10  10  10  01  01  01  01  11  11  11  11

# Calculating permutations

Where $p$ is the total number of pixels and $c$ is the maximum number of colors, $p$ pixels can have $c$ possible values that can be combined with each value of each other pixel in the image, or $p^{c}$.

In [39]:
def num_permutations(num_pixels, num_colors):
    return num_pixels ** num_colors

We can test this function out with a few examples:

In [40]:
# A 2x2 image with 2 colors
num_permutations(2*2, 2)

16

In [41]:
# A 2x2 image with 4 colors
num_permutations(2*2, 4)

256

In [42]:
# A 4x4 image with 2 colors
num_permutations(4*4, 2)

256

# Generating pixel data

If you look at the order of the permutations in our example 2x2 image you'll notice that the pixels increment linearly and sequentially, making it trivial to devise an algorithm to determine their values:

    00  10  01  11  00  10  01  11  00  10  01  11  00  10  01  11
    00  00  00  00  10  10  10  10  01  01  01  01  11  11  11  11

When generated sequentially like this, each pixel cycles through all of its color values in succession before wrapping back around to zero when its next neighbour is incremented. This continues until every pixel is equal to the maximum color value.

The first pixel increments every iteration, but the second pixel increments only when the first pixel has completed its cycle and reset to zero and so it takes two iterations to change value. The third pixel increments only when the _second_ pixel has completed its cycle and reset to zero and so it takes _four_ iterations to change value.

So where $i$ is the index position of a given pixel and $c$ is the maximum number of colors, the rate at which a pixel increments is determined by $c^{i}$, and we can use this to determine what value a pixel will have after a given number of iterations.

In [48]:
# The first pixel increments every 1 permutation
2 ** 0

1

In [49]:
# The second pixel increments every 2 permutations
2 ** 1

2

In [50]:
# The third pixel increments every 4 permutations
2 ** 2

4

So where $p$ is the index of the iteration in the sequence and $c$ is the maximum number of colors, the value of the pixel at index position $i$ can be determined by the integer value of $int((\frac{p}{c^{i}}) mod c)$.

In [62]:
def get_pixel(iteration, pixel, num_pixels, num_colors):
    return int((iteration / (num_colors ** pixel)) % num_colors)

We can test this out by calculating the values of the 6th iteration of our 2x2 example (1010):

In [63]:
get_pixel(5, 0, 2*2, 2)

1

In [64]:
get_pixel(5, 1, 2*2, 2)

0

In [65]:
get_pixel(5, 2, 2*2, 2)

1

In [66]:
get_pixel(5, 3, 2*2, 2)

0

# Generating image data

It's pretty trivial now to create a function to iterate through all the pixels of a given iteration and return their values:

In [67]:
def get_image(iteration, num_pixels, num_colors):
    return list(map(lambda i: get_pixel(iteration, i, num_pixels, num_colors), range(0, num_pixels)))

We can test these out by grabbing some permutations from our 2x2 example image for comparison:

In [68]:
# Get the first image
get_image(0, 2*2, 2)

[0, 0, 0, 0]

In [69]:
# Get the second image
get_image(1, 2*2, 2)

[1, 0, 0, 0]

In [70]:
# Get the third image
get_image(2, 2*2, 2)

[0, 1, 0, 0]

# Generating an image

Creating a function to create an image is pretty simple:

In [132]:
from PIL import Image

def generate_image(data, image_size, num_colors):
    image = Image.new('RGB', (image_size, image_size))
    
    for y in range(0, image_size):
        for x in range(0, image_size):
            color = int((255 / num_colors) * data[y * image_size + x])
            image.putpixel((x, y), (color, color, color))
    
    return image

# Putting it all together

Now we've got all the parts, we can try generating an _actual_ image:

In [134]:
from random import random

image_size = 16
num_colors = 2

# Get the number of permutations
permutations = num_permutations(num_colors, image_size * image_size)

# Get a random iteration index
iteration = int((permutations / 1) * random())

# Get the data for that permutation
data = get_image(iteration, image_size * image_size, num_colors)

# Generate an image
generate_image(data, image_size, num_colors)

# The problem

Things start to go awry as you increase the size and color depth of the image. A tiny image of just 32x32 pixels and 8 colors has an incredible number of permutations:

In [75]:
(32 * 32) ** 8

1208925819614629174706176

And if we stretch for something _barely_ usable, this happens:

In [76]:
(128 * 128) ** 8

5192296858534827628530496329220096

But really, our bare minimum is more this:

In [79]:
(256 * 256) ** 256

1044388881413152506691752710716624382579964249047383780384233483283953907971557456848826811934997558340890106714439262837987573438185793607263236087851365277945956976543709998340361590134383718314428070011855946226376318839397712745672334684344586617496807908705803704071284048740118609114467977783598029006686938976881787785946905630190260940599579453432823469303026696443059025015972399867714215541693835559885291486318237914434496734087811872639496475100189041349008417061675093668333850551032972088269550769983616369411933015213796825837188091833656751221318492846368125550225998300412344784862595674492194617023806505913245610825731835380087608622102834270197698202313169017678006675195485079921636419370285375124784014907159135459982790513399611551794271106831134090584272884279791554849782954323534517065223269061394905987693002122963395687782878948440616007412945674919823050571642377154816321380631045902916136926708342856440730447899971901781465763473223850267253059899795996090799469201774

And ideally, we'd be able to do this:

In [82]:
(512 * 512) ** 512

1960830435266894322156539015083909550871875860382118335085727055253277825836538973132036028341386877890063590564638661218272958143533989172469170582635293991946517116704191729729501018887561528561979061199679112074196578288359970451231493309531382213825518265192957801481848302576280475409650352336107952264533454052199103766379175150196274451042330021035058774192298703214134389164953066448444033146483301629755998386419833465661969283956338813588851525864148809052881908619330856495434373926683963853153861488276758703092055309189870243218929647154797233566595639559176925078185557911025274361886571529847680150156580439765217169746447257682028680026157221615520776910748302079142377873323855161511341545388155356404969082629981623725247870844566857148352510072932530604362505361531975420664533753584017837511855498034875918420884074025277017561382997989517664989597201036493533942611858781362041975130491088788596943191686787350245270205239132623344250052954199439290526563834984074418657506792139

# The goal

Ideally we want to be able to generate images of _at least_ 256x256 pixels with 256 colors, but preferably as high as 512x512 with 512 colors. I'll let you do the mathematics on the number of permutations. Try changing some of the variables in the example image generation code above and see what happens.

The puzzle is simply to devise a way to allow _all_ permutations to be referenced and generated by an index.